In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/NYU/Courses/DSGA1011/Project/output_QA/arc_nos_cleaned/arc_nos_cleaned

/content/drive/MyDrive/NYU/Courses/DSGA1011/Project/output_QA/arc_nos_cleaned/arc_nos_cleaned


In [ ]:
cd /content/drive/MyDrive/NYU/Courses/DSGA1011/Project/output_1011_parameter/output_1011_parameter/truthfuqa_noc_cleaned/truthfuqa_noc_cleaned

/content/drive/MyDrive/NYU/Courses/DSGA1011/Project/output_1011_parameter/output_1011_parameter/truthfuqa_noc_cleaned/truthfuqa_noc_cleaned


In [ ]:
import pandas as pd
from pandas._config.config import option_context
import numpy as np

In [ ]:
filename = "truthfulQA_16_GPT_Cleaned.csv"

In [ ]:
df = pd.read_csv(filename, encoding='latin-1')

df['GPT inferred confidence level'] = pd.to_numeric(df['GPT inferred confidence level'], errors='coerce')
#df['Confidence Level'] = pd.to_numeric(df['Confidence Level'], errors='coerce')
df = df.dropna()
print(df.shape[0])

count_1 = df[(df['GPT inferred answer'] == df['Correct Answer'])].shape[0]
count_2 = df[(df['GPT inferred answer'] != df['Correct Answer']) & (df['GPT inferred confidence level'] >= 80)].shape[0]
#count_2 = df[(df['GPT inferred answer'] != df['answer']) & (df['Confidence Level'] >= 80)].shape[0]
print(count_1 / len(df))
print(count_2 / len(df))
def calculate_ece(predictions, true_labels, confidences, threshold=80, num_bins=10):
    confidences = confidences / 100
    threshold = threshold / 100
    incorrect = np.where(predictions != true_labels)
    i_confidences = confidences[incorrect]

    bin_edges = np.linspace(0, 1, num_bins+1)
    bin_indices_o = np.digitize(confidences, bin_edges, right=True)
    bin_indices_oi = np.digitize(i_confidences, bin_edges, right=True)

    ece = 0
    total_instances = len(bin_indices_o)

    for bin_idx in range(0, num_bins + 1):
        instances_in_bin = np.where(bin_indices_o == bin_idx)
        if len(instances_in_bin[0]) == 0:
            continue
        avg_confidence = np.mean(confidences[instances_in_bin])
        observed_accuracy = np.count_nonzero(true_labels[instances_in_bin] == predictions[instances_in_bin]) / len(instances_in_bin[0])

        ece += (len(instances_in_bin[0]) / total_instances) * np.abs(avg_confidence - observed_accuracy)

    return ece

df = df[pd.notnull(df['GPT inferred confidence level'])]
predictions = df['GPT inferred answer'].values
true_labels = df['Correct Answer'].values
confidences = df['GPT inferred confidence level'].values
#confidences = df['Confidence Level'].values

ece = calculate_ece(predictions, true_labels, confidences)
print("ECE for overconfident predictions:", ece)

157
0.27388535031847133
0.5605095541401274
ECE for overconfident predictions: 0.6070063694267516
